# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
#!/usr/bin/env python
import re
import sys
import pickle
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score, recall_score, f1_score,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
def load_data(dbname, tablename):
    """ Load method
    input:
        dbname: database name 
        tablename: database table name
    output:
        X = input ,Y = target dataframe 
    """    
    engine = create_engine('sqlite:///{}'.format(dbname))
    df = pd.read_sql_table(tablename, con=engine)
    X = df['message']
    Y = df.drop(['id','original','message', 'genre'], axis=1)
    return X, Y

In [3]:
X, Y = load_data('disaster.db', 'disaster_messages')

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(string):
    """
    tokenize an clean non english words. then , lemmatize the words
    :input : long sentences
    :outpout: tokenized, lemmetized words
    """
    string = re.sub('[^(a-zA-Z0-9)]',' ',string)
    tokenized = word_tokenize(string)
     # Remove Stopwords
    removed_words = [w for w in tokenized if w not in stopwords.words('english')]
    
    tokenized_cleaned_lemmed = [WordNetLemmatizer().lemmatize(w) for w in tokenized]
    tokenized_cleaned_lemmed_stemm = [PorterStemmer().stem(word).lower() for word in tokenized_cleaned_lemmed]
    
    return tokenized_cleaned_lemmed_stemm

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=23)

In [7]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [8]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def scores(y_pred, y_true):
    for i, column in enumerate(y_true.columns): 
        print('report for %s'%(column))
        print(classification_report(y_true.iloc[:,i], y_pred[:,i]))

In [10]:
scores(y_pred, Y_test)

report for related
             precision    recall  f1-score   support

          0       0.62      0.37      0.47      1824
          1       0.83      0.93      0.87      5981
          2       0.47      0.27      0.34        60

avg / total       0.78      0.80      0.78      7865

report for request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      6462
          1       0.87      0.37      0.52      1403

avg / total       0.88      0.88      0.86      7865

report for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7833
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      7865

report for aid_related
             precision    recall  f1-score   support

          0       0.73      0.87      0.79      4580
          1       0.74      0.54      0.63      3285

avg / total       0.73      0.73      0.72      7865

report fo

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
X_test.shape

(7865,)

In [12]:
Y_test.shape

(7865, 36)

In [13]:
y_pred.shape

(7865, 36)

In [14]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred])))

0.384361093452


In [15]:
print(classification_report(Y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=Y_test.columns))

                        precision    recall  f1-score   support

               related       0.87      0.37      0.52      1403
               request       0.00      0.00      0.00        32
                 offer       0.74      0.54      0.63      3285
           aid_related       0.64      0.10      0.18       622
          medical_help       0.71      0.08      0.14       398
      medical_products       0.74      0.08      0.14       223
     search_and_rescue       0.00      0.00      0.00       152
              security       0.67      0.10      0.17       248
              military       0.00      0.00      0.00         0
           child_alone       0.82      0.25      0.38       538
                 water       0.90      0.33      0.49       912
                  food       0.80      0.16      0.27       696
               shelter       0.56      0.08      0.14       121
              clothing       0.50      0.01      0.02       198
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [53]:
# model pipeline
grid_search_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, stop_words='english')),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', MultiOutputClassifier((AdaBoostClassifier())))
])

parameters = {
'clf__estimator__n_estimators' : [5,10],
'clf__estimator__learning_rate' : [0.5],
}
grid_search_clf = GridSearchCV(grid_search_pipeline, parameters)

In [54]:
grid = grid_search_clf.fit(X_train, Y_train)

In [55]:
print("Grid Search Results - Best Parameters")
print(grid.best_score_)
print(grid.best_params_)

Grid Search Results - Best Parameters
0.214211759577
{'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 10}


In [57]:
model_test_pred=grid.predict(X_test)

In [58]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test.iloc[:,1:].values, np.array([x[1:] for x in model_test_pred])))

0.427336300064


In [9]:
# try to find best
improved_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, stop_words='english')),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42, n_estimators=60,max_depth=4,n_jobs=4,verbose=3 )))
])


In [10]:
improved_pipeline.fit(X_train, Y_train)
y_pred_improved = improved_pipeline.predict(X_test)

building tree 1 of 60
building tree 2 of 60
building tree 3 of 60
building tree 4 of 60
building tree 5 of 60building tree 6 of 60building tree 7 of 60building tree 8 of 60



building tree 9 of 60
building tree 10 of 60building tree 11 of 60building tree 12 of 60


building tree 13 of 60
building tree 14 of 60building tree 15 of 60building tree 16 of 60


building tree 17 of 60
building tree 18 of 60building tree 19 of 60building tree 20 of 60building tree 21 of 60



building tree 22 of 60building tree 23 of 60building tree 24 of 60
building tree 25 of 60


building tree 26 of 60building tree 27 of 60building tree 28 of 60
building tree 29 of 60


building tree 30 of 60building tree 31 of 60
building tree 32 of 60building tree 33 of 60


building tree 34 of 60building tree 35 of 60
building tree 36 of 60

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.2s


building tree 37 of 60


building tree 38 of 60building tree 39 of 60
building tree 40 of 60building tree 41 of 60
building tree 42 of 60

building tree 43 of 60
building tree 44 of 60
building tree 45 of 60
building tree 46 of 60
building tree 47 of 60building tree 48 of 60


building tree 49 of 60
building tree 50 of 60building tree 51 of 60building tree 52 of 60

building tree 53 of 60

building tree 54 of 60building tree 55 of 60building tree 56 of 60
building tree 57 of 60


building tree 58 of 60building tree 59 of 60building tree 60 of 60




[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:    0.4s finished


building tree 1 of 60
building tree 2 of 60
building tree 3 of 60
building tree 4 of 60
building tree 5 of 60building tree 6 of 60building tree 7 of 60building tree 8 of 60



building tree 9 of 60building tree 10 of 60
building tree 11 of 60
building tree 12 of 60
building tree 13 of 60

building tree 14 of 60building tree 15 of 60building tree 16 of 60
building tree 17 of 60

building tree 18 of 60

building tree 19 of 60building tree 20 of 60
building tree 21 of 60
building tree 22 of 60
building tree 23 of 60building tree 24 of 60

building tree 25 of 60

building tree 26 of 60
building tree 27 of 60building tree 28 of 60
building tree 29 of 60building tree 30 of 60

building tree 31 of 60building tree 32 of 60


building tree 33 of 60
building tree 34 of 60building tree 35 of 60
building tree 36 of 60
building tree 37 of 60
building tree 38 of 60



...

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:    0.3s finished


building tree 44 of 60
building tree 45 of 60
building tree 46 of 60
building tree 47 of 60
building tree 48 of 60
building tree 49 of 60building tree 50 of 60building tree 51 of 60

building tree 52 of 60
building tree 53 of 60
building tree 54 of 60building tree 55 of 60
building tree 56 of 60


building tree 57 of 60
building tree 58 of 60building tree 59 of 60

building tree 60 of 60
building tree 1 of 60
building tree 2 of 60
building tree 3 of 60
building tree 4 of 60
building tree 5 of 60building tree 6 of 60building tree 7 of 60building tree 8 of 60



building tree 9 of 60building tree 10 of 60building tree 11 of 60building tree 12 of 60



building tree 13 of 60building tree 14 of 60building tree 15 of 60building tree 16 of 60
building tree 17 of 60



building tree 18 of 60building tree 19 of 60building tree 20 of 60


building tree 21 of 60building tree 22 of 60
building tree 23 of 60building tree 24 of 60
building tree 25 of 60


building tree 26 of 60building tree 27 of 6

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:    0.3s finished



building tree 38 of 60building tree 39 of 60building tree 40 of 60
building tree 41 of 60

building tree 42 of 60

building tree 43 of 60building tree 44 of 60building tree 45 of 60


building tree 46 of 60
building tree 47 of 60
building tree 48 of 60building tree 49 of 60
building tree 50 of 60

building tree 51 of 60
building tree 52 of 60building tree 53 of 60

building tree 54 of 60
building tree 55 of 60
building tree 56 of 60building tree 57 of 60building tree 58 of 60
building tree 59 of 60
building tree 60 of 60




[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred_improved])))

0.433693579148


In [14]:
print(classification_report(Y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred_improved]), target_names=Y_test.columns))

                        precision    recall  f1-score   support

               related       0.00      0.00      0.00      1403
               request       0.00      0.00      0.00        32
                 offer       1.00      0.01      0.02      3285
           aid_related       0.00      0.00      0.00       622
          medical_help       0.00      0.00      0.00       398
      medical_products       0.00      0.00      0.00       223
     search_and_rescue       0.00      0.00      0.00       152
              security       0.00      0.00      0.00       248
              military       0.00      0.00      0.00         0
           child_alone       0.00      0.00      0.00       538
                 water       0.00      0.00      0.00       912
                  food       0.00      0.00      0.00       696
               shelter       0.00      0.00      0.00       121
              clothing       0.00      0.00      0.00       198
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

improve model with hyper parameter tuning

In [95]:
# model pipeline
improved_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, stop_words='english')),
    ('tfidf', TfidfTransformer(norm='l2')),
    #('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=70,max_depth=4,n_jobs=4,verbose=3 )))
    ('clf', MultiOutputClassifier((AdaBoostClassifier(learning_rate=0.50, n_estimators=70,random_state=42))))
])


improved_pipeline.fit(X_train, Y_train)
y_pred_improved = improved_pipeline.predict(X_test)

In [96]:
# Calculate the accuracy for each of them.
for i in range(len(Y_test.columns)):
    print("Category:", Y_test.columns[i],"\n", classification_report(Y_test.iloc[:, i].values, y_pred_improved[:, i]))
    print('Accuracy of %25s: %.2f' %(Y_test.columns[i], accuracy_score(Y_test.iloc[:, i].values, y_pred_improved[:,i])))

Category: related 
              precision    recall  f1-score   support

          0       0.73      0.04      0.07      1824
          1       0.77      1.00      0.87      5981
          2       1.00      0.10      0.18        60

avg / total       0.76      0.77      0.68      7865

Accuracy of                   related: 0.77
Category: request 
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      6462
          1       0.82      0.44      0.58      1403

avg / total       0.88      0.88      0.87      7865

Accuracy of                   request: 0.88
Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7833
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      7865

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

          0       0.75      0.89  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [97]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred_improved])))

0.427590591227


In [98]:
print(classification_report(Y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred_improved]), target_names=Y_test.columns))

                        precision    recall  f1-score   support

               related       0.82      0.44      0.58      1403
               request       0.00      0.00      0.00        32
                 offer       0.79      0.58      0.67      3285
           aid_related       0.65      0.19      0.30       622
          medical_help       0.73      0.24      0.36       398
      medical_products       0.74      0.14      0.23       223
     search_and_rescue       0.00      0.00      0.00       152
              security       0.67      0.23      0.34       248
              military       0.00      0.00      0.00         0
           child_alone       0.74      0.67      0.70       538
                 water       0.81      0.72      0.76       912
                  food       0.79      0.52      0.62       696
               shelter       0.72      0.48      0.58       121
              clothing       0.63      0.21      0.31       198
                 money       0.77      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning:

labels size, 35, does not match size of target_names, 36

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



### 9. Export your model as a pickle file

In [93]:
import pickle
def save_model(model, model_file_name):
    pickle.dump(model, open(model_file_name, 'wb'))
def load_model(model, model_file_name):
    pickle.load(open(model_file_name, 'rb'))

In [94]:
save_model(y_pred_improved, "model.pick")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.